In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils.finance_metrics import annualized_return
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, median_absolute_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from keras.models import Sequential
from keras.layers import GRU, SimpleRNN, Dense, Dropout
from keras.metrics import MeanSquaredError, RootMeanSquaredError
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
def build_RNN():
    # Generate the model architecture
    rnn_model = Sequential()
    rnn_model.add(GRU(128, input_dim=44, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    rnn_model.add(SimpleRNN(128))
    rnn_model.add(Dense(64, activation='relu'))
    rnn_model.add(Dropout(0.3))
    rnn_model.add(Dense(32, activation='relu'))
    rnn_model.add(Dropout(0.3))
    rnn_model.add(Dense(16, activation='relu'))
    rnn_model.add(Dropout(0.3))
    rnn_model.add(Dense(1))
    # Compile the model
    rnn_model.compile(optimizer='adam', loss='mean_squared_error',
                  metrics=[MeanSquaredError(), RootMeanSquaredError()])
    return rnn_model

In [ ]:
data_path = r'data/processed'
data_filenames = os.listdir(data_path)

models = {}
feature_transformers = {}
feature_scalers = {}
for data_file in data_filenames:
    pairs = data_file[:-4].split('-')
    pair = (pairs[0], pairs[1])
    print(pair)

    pair_df = pd.read_pickle(os.path.join(data_path, data_file))
    X_train = pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")
    y_train = pair_df.loc["2000-01-01":"2014-12-31", ["Return Diff (t+1)"]]
    X_test = pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")
    y_test = pair_df.loc["2015-01-01":, ["Return Diff (t+1)"]]

    # fill invalid values with 1 (for ratios) or 0 (for differences)
    X_train[X_train.isin([-np.inf, np.inf, np.nan])] = 1
    y_train[y_train.isin([-np.inf, np.inf, np.nan])] = 0
    X_test[X_test.isin([-np.inf, np.inf, np.nan])] = 1
    y_test[y_test.isin([-np.inf, np.inf, np.nan])] = 0

    # add in quantiles as additional feature columns
    quantile_transformer = QuantileTransformer()
    quantile_transformer.fit(X_train); feature_transformers[pair] = quantile_transformer
    X_train.loc[:, [col + "_QUANTILE" for col in X_train.columns]] = pd.DataFrame(
        pd.DataFrame(quantile_transformer.transform(X_train), index=X_train.index,
                     columns=[col + "_QUANTILE" for col in X_train.columns]))
    X_test.loc[:, [col + "_QUANTILE" for col in X_test.columns]] = pd.DataFrame(
        pd.DataFrame(quantile_transformer.transform(X_test), index=X_test.index,
                     columns=[col + "_QUANTILE" for col in X_test.columns]))

    # fill invalid values with 1 (for ratios) or 0 (for differences)
    X_train[X_train.isin([-np.inf, np.inf, np.nan])] = 1
    y_train[y_train.isin([-np.inf, np.inf, np.nan])] = 0
    X_test[X_test.isin([-np.inf, np.inf, np.nan])] = 1
    y_test[y_test.isin([-np.inf, np.inf, np.nan])] = 0

    # scale features
    X_scaler = StandardScaler()
    X_scaler.fit(X_train); feature_scalers[pair] = X_scaler
    X_train = pd.DataFrame(X_scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(X_scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    assert not any([any(arr) for arr in np.array(np.isinf(X_train))])
    assert not any([any(arr) for arr in np.array(np.isnan(X_train))])
    assert not any([any(arr) for arr in np.array(np.isinf(X_test))])
    assert not any([any(arr) for arr in np.array(np.isnan(X_test))])
    assert not any([any(arr) for arr in np.array(np.isinf(y_train))])
    assert not any([any(arr) for arr in np.array(np.isnan(y_train))])
    assert not any([any(arr) for arr in np.array(np.isinf(y_test))])
    assert not any([any(arr) for arr in np.array(np.isnan(y_test))])

    split = PredefinedSplit(test_fold=[0 if v else -1 for v in X_train.index < '2012-01-01'])
    models[pair] = {
        'Random Forest': GridSearchCV(estimator=RandomForestRegressor(bootstrap=True), param_grid={
            "n_estimators": [n for n in range(50, 150, 25)]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Adaptive Boost': GridSearchCV(estimator=AdaBoostRegressor(), param_grid={
            "n_estimators": [n for n in range(50, 250, 50)],
            "loss": ["linear", "exponential"]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Gradient Boost': GridSearchCV(estimator=GradientBoostingRegressor(loss="huber"), param_grid={
            "n_estimators": [n for n in range(50, 250, 50)]
        }, cv=split, scoring="explained_variance", n_jobs=-1),
        'Recurrent Neural Net': GridSearchCV(estimator=KerasRegressor(build_fn=build_RNN, verbose=False), param_grid={
            "nb_epoch": [100, 500, 1000],  # TBD
            "batch_size": [128, 256]
        }, cv=split, scoring="explained_variance", n_jobs=-1)
    }

    print(f"==================================================\n"
          f"Results for pair {pair}:\n"
          f"- Return Difference Stats:\n"
          f"  - mean: {float(np.mean(y_test)):.06f}\n"
          f"  - std: {float(np.std(y_test)):.06f}\n"
          f"  - quantiles [0.00, 0.25, 0.50, 0.75, 1.00]:\n"
          f"    {[np.round(i, decimals=6) for i in np.quantile(y_test, [0, .25, .50, .75, 1])]}")
    pred_average = None
    for model_type, model in models[pair].items():
        model.fit(X_train, np.array(y_train).ravel())
        pred = model.predict(X_test)
        if pred_average is None: pred_average = pred / 4
        else: pred_average = pred_average + np.array(pred / 4)
        mse = mean_squared_error(np.array(y_test), np.array(pred).ravel())
        mae = median_absolute_error(np.array(y_test), np.array(pred).ravel())
        print(f"- Model: {model_type}\n"
              f"  - best parameters: {model.best_params_}\n"
              f"  - MSE: {mse:.06f}\n"
              f"  - MAE: {mae:.06f}\n"
              f"--------------------------------------------------")
    mse = mean_squared_error(np.array(y_test), np.array(pred_average).ravel())
    mae = median_absolute_error(np.array(y_test), np.array(pred_average).ravel())
    print(f"- Ensemble of Random Forest, Adaptive Boost, Gradient Boost, and Neural Net\n"
          f"  - MSE: {mse:.06f}\n"
          f"  - MAE: {mae:.06f}\n"
          f"--------------------------------------------------")
    print(f"==================================================\n")


In [ ]:
conf_mat_all = None
for data_file in data_filenames:
    pairs = data_file[:-4].split('-')
    pair = (pairs[0], pairs[1])
    print(pair)

    pair_df = pd.read_pickle(os.path.join(data_path, data_file))
    X_train = pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")
    y_train = pair_df.loc["2000-01-01":"2014-12-31", ["Return Diff (t+1)"]]
    X_test = pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")
    y_test = pair_df.loc["2015-01-01":, ["Return Diff (t+1)"]]

    # fill invalid values with 1 (for ratios) or 0 (for differences)
    X_train[X_train.isin([-np.inf, np.inf, np.nan])] = 1
    y_train[y_train.isin([-np.inf, np.inf, np.nan])] = 0
    X_test[X_test.isin([-np.inf, np.inf, np.nan])] = 1
    y_test[y_test.isin([-np.inf, np.inf, np.nan])] = 0

    # add in quantiles as additional feature columns
    quantile_transformer = feature_transformers[pair]
    X_train.loc[:, [col + "_QUANTILE" for col in X_train.columns]] = pd.DataFrame(
        pd.DataFrame(quantile_transformer.transform(X_train), index=X_train.index,
                     columns=[col + "_QUANTILE" for col in X_train.columns]))
    X_test.loc[:, [col + "_QUANTILE" for col in X_test.columns]] = pd.DataFrame(
        pd.DataFrame(quantile_transformer.transform(X_test), index=X_test.index,
                     columns=[col + "_QUANTILE" for col in X_test.columns]))

    # fill invalid values with 1 (for ratios) or 0 (for differences)
    X_train[X_train.isin([-np.inf, np.inf, np.nan])] = 1
    X_test[X_test.isin([-np.inf, np.inf, np.nan])] = 1

    # scale features
    X_scaler = feature_scalers[pair]
    X_train = pd.DataFrame(X_scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(X_scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    assert not any([any(arr) for arr in np.array(np.isinf(X_train))])
    assert not any([any(arr) for arr in np.array(np.isnan(X_train))])
    assert not any([any(arr) for arr in np.array(np.isinf(X_test))])
    assert not any([any(arr) for arr in np.array(np.isnan(X_test))])
    assert not any([any(arr) for arr in np.array(np.isinf(y_train))])
    assert not any([any(arr) for arr in np.array(np.isnan(y_train))])
    assert not any([any(arr) for arr in np.array(np.isinf(y_test))])
    assert not any([any(arr) for arr in np.array(np.isnan(y_test))])

    print(f"==================================================\n"
          f"Results for pair {pair}:")
    pred_ensemble = None
    for model_type, model in models[pair].items():
        pred = [-1 if i < 0 else 1 for i in np.array(model.predict(X_test)).ravel()]
        if pred_ensemble is None: pred_ensemble = np.array(pred) / 4
        else: pred_ensemble = pred_ensemble + (np.array(pred) / 4)

        conf_mat = confusion_matrix(
            [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
            [0 if i < 0 else 1 for i in np.array(pred).ravel()])
        accuracy = accuracy_score(
            [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
            [0 if i < 0 else 1 for i in np.array(pred).ravel()])
        precision = precision_score(
            [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
            [0 if i < 0 else 1 for i in np.array(pred).ravel()])
        recall = recall_score(
            [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
            [0 if i < 0 else 1 for i in np.array(pred).ravel()])
        f1 = f1_score(
            [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
            [0 if i < 0 else 1 for i in np.array(pred).ravel()])

        print(f"- Model: {model_type}\n"
              f"  - Confusion Matrix:\n{conf_mat}\n"
              f"  - Accuracy: {accuracy:.06f}\n"
              f"  - Recall: {recall:.06f}\n"
              f"  - Precision: {precision:.06f}\n"
              f"  - F1-Score: {f1:.06f}\n"
              f"--------------------------------------------------")

    conf_mat = confusion_matrix(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    accuracy = accuracy_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    precision = precision_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    recall = recall_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    f1 = f1_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])

    if conf_mat_all is None: conf_mat_all = np.array(conf_mat)
    else: conf_mat_all = conf_mat_all + np.array(conf_mat)

    print(f"- Ensemble of Random Forest, Adaptive Boost, Gradient Boost, and Neural Net\n"
          f"  - Confusion Matrix:\n{conf_mat}\n"
          f"  - Accuracy: {accuracy:.06f}\n"
          f"  - Recall: {recall:.06f}\n"
          f"  - Precision: {precision:.06f}\n"
          f"  - F1-Score: {f1:.06f}\n"
          f"--------------------------------------------------")
    print(f"==================================================\n")


print(f"- Confusion Matrix of ensemble prediction across all pairs\n{conf_mat_all}\n"
      f"  - Accuracy: {(conf_mat_all[0][0]+conf_mat_all[1][1]) / np.sum(conf_mat_all):.06f}\n"
      f"--------------------------------------------------")
print(f"==================================================\n")

In [ ]:
def signal_to_multiplier(signal, long):
    if           signal == -1.0: return -0.50 * long
    elif -1.0 <  signal <= -0.5: return -0.25 * long
    elif -0.5 <  signal <= -0.0: return -0.10 * long
    elif  0.0 <= signal <   0.5: return  0.10 * long
    elif  0.5 <= signal <   1.0: return  0.25 * long
    elif         signal ==  1.0: return  0.50 * long
    else: raise ValueError("Out of range signal provided")

returns = {}
for data_file in data_filenames:
    pairs = data_file[:-4].split('-')
    pair = (pairs[0], pairs[1])
    print(pair)

    returns[pair] = pd.read_pickle(f"data/raw/{pair[0]}.zip").loc["2015-01-01":, ["Simple Return"]]
    returns[pair][pair[1]] = pd.read_pickle(f"data/raw/{pair[1]}.zip").loc["2015-01-01":, ["Simple Return"]]
    returns[pair].columns = pair

    pair_df = pd.read_pickle(os.path.join(data_path, data_file))
    X_train = pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")
    y_train = pair_df.loc["2000-01-01":"2014-12-31", ["Return Diff (t+1)"]]
    X_test = pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")
    y_test = pair_df.loc["2015-01-01":, ["Return Diff (t+1)"]]

    # fill invalid values with 1 (for ratios) or 0 (for differences)
    X_train[X_train.isin([-np.inf, np.inf, np.nan])] = 1
    y_train[y_train.isin([-np.inf, np.inf, np.nan])] = 0
    X_test[X_test.isin([-np.inf, np.inf, np.nan])] = 1
    y_test[y_test.isin([-np.inf, np.inf, np.nan])] = 0

    # add in quantiles as additional feature columns
    quantile_transformer = feature_transformers[pair]
    X_train.loc[:, [col + "_QUANTILE" for col in X_train.columns]] = pd.DataFrame(
        pd.DataFrame(quantile_transformer.transform(X_train), index=X_train.index,
                     columns=[col + "_QUANTILE" for col in X_train.columns]))
    X_test.loc[:, [col + "_QUANTILE" for col in X_test.columns]] = pd.DataFrame(
        pd.DataFrame(quantile_transformer.transform(X_test), index=X_test.index,
                     columns=[col + "_QUANTILE" for col in X_test.columns]))

    # fill invalid values with 1 (for ratios) or 0 (for differences)
    X_train[X_train.isin([-np.inf, np.inf, np.nan])] = 1
    y_train[y_train.isin([-np.inf, np.inf, np.nan])] = 0
    X_test[X_test.isin([-np.inf, np.inf, np.nan])] = 1
    y_test[y_test.isin([-np.inf, np.inf, np.nan])] = 0

    # scale features and target column
    X_scaler = feature_scalers[pair]
    X_train = pd.DataFrame(X_scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(X_scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    assert not any([any(arr) for arr in np.array(np.isinf(X_train))])
    assert not any([any(arr) for arr in np.array(np.isnan(X_train))])
    assert not any([any(arr) for arr in np.array(np.isinf(X_test))])
    assert not any([any(arr) for arr in np.array(np.isnan(X_test))])
    assert not any([any(arr) for arr in np.array(np.isinf(y_train))])
    assert not any([any(arr) for arr in np.array(np.isnan(y_train))])
    assert not any([any(arr) for arr in np.array(np.isinf(y_test))])
    assert not any([any(arr) for arr in np.array(np.isnan(y_test))])

    print(f"==================================================\n"
          f"Results for pair {pair}:")
    pred_ensemble = None
    for model_type, model in models[pair].items():
        pred = [-1 if i < 0 else 1 for i in np.array(model.predict(X_test)).ravel()]
        if pred_ensemble is None: pred_ensemble = np.array(pred) / 4
        else: pred_ensemble = pred_ensemble + (np.array(pred) / 4)
        returns[pair][model_type] = pred
    returns[pair]["Ensemble"] = np.mean(
        returns[pair].loc[:, ["Random Forest", "Adaptive Boost", "Gradient Boost", "Recurrent Neural Net"]], axis=1)
    returns[pair]["Ensemble"] = pred_ensemble
    returns[pair][pair[0] + "_SIGNAL"] = [signal_to_multiplier(sig,  1) for sig in pred_ensemble]
    returns[pair][pair[1] + "_SIGNAL"] = [signal_to_multiplier(sig, -1) for sig in pred_ensemble]
    returns[pair][pair[0] + "_ADJRET"] = returns[pair][pair[0]] * returns[pair][pair[0] + "_SIGNAL"]
    returns[pair][pair[1] + "_ADJRET"] = returns[pair][pair[1]] * returns[pair][pair[1] + "_SIGNAL"]
    returns[pair]["PAIRS_TRADE_RET"] = np.mean(returns[pair][[p+"_ADJRET" for p in pair]], axis=1)
    ann_ret = annualized_return(returns[pair].loc[:, [pair[0],pair[1],"PAIRS_TRADE_RET"]])

    conf_mat = confusion_matrix(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    accuracy = accuracy_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    precision = precision_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    recall = recall_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])
    f1 = f1_score(
        [0 if i < 0 else 1 for i in y_test.to_numpy().ravel()],
        [0 if i < 0 else 1 for i in np.array(pred_ensemble).ravel()])

    print(f"- Ensemble of Random Forest, Adaptive Boost, Gradient Boost, and Recurrent Neural Net\n"
          f"  - Confusion Matrix:\n{conf_mat}\n"
          f"  - Accuracy: {accuracy:.06f}\n"
          f"  - Recall: {recall:.06f}\n"
          f"  - Precision: {precision:.06f}\n"
          f"  - F1-Score: {f1:.06f}\n"
          f"- Annualized Return:\n"
          f"  - {pair[0]}: {ann_ret[pair[0]]:0.06f}\n"
          f"  - {pair[1]}: {ann_ret[pair[1]]:0.06f}\n"
          f"  - Pairs Trade: {ann_ret['PAIRS_TRADE_RET']:0.06f}\n"
          f"--------------------------------------------------")
    print(f"==================================================\n")

    # Compute the correlation matrix
    corr = returns[pair].loc[:, [pair[0],pair[1],"PAIRS_TRADE_RET"]].corr()
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))
    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    # Draw the heatmap with the mask and correct aspect ratio
    heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, annot=True,
                          square=True, linewidths=.5, cbar_kws={"shrink": .5})
    heatmap.set_title(f"{pair[0]}-{pair[1]} Pairs Trade Correlation")
    fig = heatmap.get_figure()
    fig.savefig(f"out/img/{pair[0]}-{pair[1]}-corr-rnn.jpeg")

    f, ax = plt.subplots(figsize=(11, 9))
    barplot = ann_ret.plot.bar()
    barplot.set_xticklabels(ann_ret.index, rotation=0)
    barplot.set_title(f"{pair[0]}-{pair[1]} Annualized Return")
    fig = barplot.get_figure()
    fig.savefig(f"out/img/{pair[0]}-{pair[1]}-return-rnn.jpeg")
